In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
from ultralytics import YOLO


MODEL = "Trained_Models/nano_segment_model.pt"
model = YOLO(MODEL) 

In [ ]:
def get_file_name_of_image_without_extension(image_path):
    import os
    image_filename_without_ext = os.path.splitext(os.path.basename(image_path))[0]
    return image_filename_without_ext

In [ ]:

def get_predicted_mask_with_specific_classes(predicted_masks,predicted_classes,defined_classes):

    side_length = 640
    height = side_length
    width = side_length
    classes_available = False

    import numpy
    mask = numpy.zeros((height,width))

    for i in range(len(predicted_classes)):
        is_in_defined_classes = False

        #checking is annoatation is present in list of defined classes
        for annotation_class in defined_classes:
            if(annotation_class == predicted_classes[i]):
                is_in_defined_classes = True
                classes_available = True
        
        if(is_in_defined_classes):
            for row_index in range(len(predicted_masks[i])):
                for col_index in range(len(predicted_masks[i][row_index])):
                    if predicted_masks[i][row_index][col_index] == 1:
                        mask[row_index][col_index] = 1


    return mask,classes_available


In [ ]:

def get_predicted_mask_from_model(filepath_of_image,defined_classes):

    result = model(filepath_of_image)
    predicted_masks = []
    predicted_classes = []

    if len(result[0]) !=0:
        predicted_masks = result[0].masks.masks.cpu().numpy()
        predicted_classes = result[0].boxes.cls.cpu().numpy()
        
    return get_predicted_mask_with_specific_classes(predicted_masks,predicted_classes,defined_classes)


In [ ]:
def get_actual_mask_from_polygon(file_name_of_label,defined_classes):

    import numpy
    from PIL import Image, ImageDraw

    side_length = 640 
    width = side_length
    height = side_length

    img = Image.new('L', (width, height), 0)

    file = open(file_name_of_label, mode = 'r', encoding = 'utf-8-sig')
    lines = file.readlines()
    file.close()
    
    classes_available = False

    for line in lines:
        line = line.split(' ')
        line = [i.strip() for i in line]
        class_of_annotation=int(line[0])
        is_in_defined_classes = False

        #checking is annoatation is present in list of defined classes
        for annotation_class in defined_classes:
            if(annotation_class == class_of_annotation):
                is_in_defined_classes = True
                classes_available = True

        if(is_in_defined_classes):
            polygon = []
            for i in range(1,len(line)):
                line[i]=float(line[i])*side_length
                polygon.append(line[i])
            ImageDraw.Draw(img).polygon(polygon, outline=1, fill=1) # creating 2D mask from segmentation labels

    mask = numpy.array(img)

    return mask,classes_available

In [ ]:
def calculateIoU(gtMask, predMask):
    # Calculate the true positives,
    # false positives, and false negatives
    tp = 0
    fp = 0
    fn = 0

    for i in range(len(gtMask)):
        for j in range(len(gtMask[0])):
            if gtMask[i][j] == 1 and predMask[i][j] == 1:
                tp += 1
            elif gtMask[i][j] == 0 and predMask[i][j] == 1:
                fp += 1
            elif gtMask[i][j] == 1 and predMask[i][j] == 0:
                fn += 1

    # Calculate IoU
    iou = tp / (tp + fp + fn)

    return iou

In [ ]:
def get_filepath_of_image_and_label(image_path): # update test folder location
    path_to_test_folder = "MASTERDATASET/test/"
    filename_without_extension = get_file_name_of_image_without_extension(image_path)
    filename_of_image = HOME + "/" + path_to_test_folder + "images/" + filename_without_extension + ".jpg"
    filename_of_label = HOME + "/" + path_to_test_folder + "labels/" + filename_without_extension + ".txt"
    return filename_of_image,filename_of_label

In [ ]:
def test_image_with_mask(mask,image_path):
    
    import cv2
    img = cv2.imread(image_path) # 640 x 640 x 3
    
    for i in range(640):
        for j in range(640):
            if mask[i][j]==0:
                img[i][j]=[0,0,0]
                
    cv2.imshow("img",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def calculateIoUforimage(image_path,defined_classes):
    
    image_path,label_path = get_filepath_of_image_and_label(image_path)

    actual_mask,actual_mask_available = get_actual_mask_from_polygon(label_path,defined_classes)
    predicted_mask,predicted_mask_available = get_predicted_mask_from_model(image_path,defined_classes)

    # test_image_with_mask(actual_mask,image_path)

    if actual_mask_available or predicted_mask_available:
        iou = calculateIoU(actual_mask,predicted_mask)
        return iou,1
    return 0,0

In [ ]:
print(model.model.names)

In [ ]:

def get_iou_threshold_value_for_given_class(defined_classes,iou_thresholds,path_to_dataset):
    
    import glob
    import numpy as np

    count_for_iou_thresholds = np.zeros(len(iou_thresholds)) 

    for image_path in glob.glob(HOME + path_to_dataset + '/test/images/*.jpg')[:]:
        iou,cnt = calculateIoUforimage(image_path,defined_classes)
        if cnt!=0:
            for i in range(len(iou_thresholds)):
                if iou>=iou_thresholds[i]:
                    count_for_iou_thresholds[i]=count_for_iou_thresholds[i]+1

    return count_for_iou_thresholds

In [ ]:
def get_class_weight(actual_mask):
    
    class_weight = 0

    for i in range(len(actual_mask)):
        for j in range(len(actual_mask[i])):
            if(actual_mask[i][j]==1):
                class_weight+=1
    
    return class_weight

In [ ]:
def calculateWeigthedIoUforimage(image_path,defined_classes):

    import numpy as np

    class_iou = np.zeros(len(defined_classes))
    class_weights = np.zeros(len(defined_classes))

    image_path,label_path = get_filepath_of_image_and_label(image_path)

    for i in range(len(defined_classes)):
        actual_mask,actual_mask_available = get_actual_mask_from_polygon(label_path,defined_classes[i])
        predicted_mask,predicted_mask_available = get_predicted_mask_from_model(image_path,defined_classes[i])
        if(actual_mask_available):
            class_weights[i] = get_class_weight(actual_mask)
        if actual_mask_available or predicted_mask_available:
            class_iou[i] = calculateIoU(actual_mask,predicted_mask)

    sum_of_class_weigths = 0
    weighted_sum_of_class_weights = 0


    for i in range(len(defined_classes)):
        weighted_sum_of_class_weights += class_weights[i]*class_iou[i]
        sum_of_class_weigths += class_weights[i]

    weigthed_iou = weighted_sum_of_class_weights/sum_of_class_weigths

    return weigthed_iou

In [ ]:

def get_weighted_iou_threshold_value_for_given_class(defined_classes,iou_thresholds,path_to_dataset):
    
    import glob
    import numpy as np

    count_for_iou_thresholds = np.zeros(len(iou_thresholds)) 

    for image_path in glob.glob(HOME + path_to_dataset + '/test/images/*.jpg')[:]:
        iou = calculateWeigthedIoUforimage(image_path,defined_classes)
        for i in range(len(iou_thresholds)):
            if iou>=iou_thresholds[i]:
                count_for_iou_thresholds[i]=count_for_iou_thresholds[i]+1

    return count_for_iou_thresholds

In [ ]:
# calculation weighted_iou

iou_thresholds = [0.00, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, .80, .85, .90]
defined_classes  = [[0],[1],[2],[3],[4],[5],[6],[7]]
path_to_dataset = '/MASTERDATASET'

count_for_iou_thresholds = get_weighted_iou_threshold_value_for_given_class(defined_classes,iou_thresholds,path_to_dataset)
print(count_for_iou_thresholds)


In [ ]:
print(count_for_iou_thresholds)

In [ ]:
# calculation global-accuracy and class-accuracy

# iou thresholds : [0.00, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, .80, .85, .90]


iou_thresholds = [0.00, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, .80, .85, .90]
classes  = [[0],[1],[2],[3],[4],[5],[6],[7],[0,1,2,3,4,5,6,7]]
overall_count_of_iou_thresholds = []
path_to_dataset = '/MASTERDATASET'

for defined_classes in classes:
    count_for_iou_thresholds = get_iou_threshold_value_for_given_class(defined_classes,iou_thresholds,path_to_dataset)
    overall_count_of_iou_thresholds.append(count_for_iou_thresholds)
    print()
    print("calculating for class: ", end=' ')
    print(defined_classes)
    print(count_for_iou_thresholds)
    print()
    